In [1]:
print(1)


1


In [2]:
import ir_datasets
import email
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import gc
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from config import *
import re
import numpy as np

/scratch/2469038g/miniconda3/envs/ryanenv2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data set-up

In [3]:
def get_sara():
    return ir_datasets.load('sara')


def dataset_to_df(sara_dataset):
    doc_ids = []
    doc_text = []
    doc_sens = []
    for doc in sara_dataset.docs_iter():
        doc_ids.append(doc.doc_id)
        doc_text.append(doc.text)
        doc_sens.append(doc.sensitivity)

    sara_dict = {'doc_id':doc_ids, 'text':doc_text, 'sensitivity':doc_sens}
    df = pd.DataFrame.from_dict(sara_dict)
    return df


def get_sample_n(data, n):
    return data.sample(n=n, random_state=1)

def get_sample_frac(data, frac):
    return data.sample(frac=frac, random_state=1)

### Pre-processing

Preprocessing 1 - Separating replies using Original Message and Forwarded tags.

In [4]:
def proc1(dataset):
    def get_orig(message, find_orig_message):
        #line_start = message.rfind('\n', 0, find_orig_message)
        line_start = find_orig_message
        base_message = message[:line_start]
        line_end = find_orig_message + 5 #message.find('\n', find_orig_message)
        orig_message = message[line_end:]
        return base_message, orig_message

    def get_separate_messages(message):
        payload = message.split('\r\n\r\n')[1]
        separate_messages = []

        find_fwd_message = payload.find('\n----- Forw')
        find_orig_message = payload.find('-----Original Message-')
        
        n = payload
        while (find_fwd_message > 0 or find_orig_message > 0):
            if (find_fwd_message > 0 and find_orig_message > 0):
                if find_fwd_message < find_orig_message:
                    b, n = get_orig(n, find_fwd_message)
                else:
                    b, n = get_orig(n, find_orig_message)
            elif (find_fwd_message > 0 and find_orig_message < 0):
                b, n = get_orig(n, find_fwd_message)
            else:
                b, n = get_orig(n, find_orig_message)
            
            find_fwd_message = n.find('\n----- Forw')
            find_orig_message = n.find('-----Original Message-')

            b = re.sub(r'\s+', ' ', b)
            separate_messages.append(b)

        n = re.sub(r'\s+', ' ', n)
        separate_messages.append(n)

        return separate_messages

    def preprocessing_dataframe(testing_sample):
        ids = []
        texts = []
        sens = []
        preproc = {}

        for s in testing_sample.iterrows():
            separate_messages = get_separate_messages(s[1].text)

            if len(separate_messages) == 1:
                ids.append(s[1].doc_id)
                texts.append(separate_messages[0])
                sens.append(s[1].sensitivity)
            else:
                for i, m in enumerate(separate_messages):
                    id_part = s[1].doc_id + '_' + str(i)
                    ids.append(id_part)
                    texts.append(m)
                    sens.append(s[1].sensitivity)

        preproc['doc_id'] = ids
        preproc['text'] = texts
        preproc['sensitivity'] = sens
        return preproc

    def get_preprocessed_sara_replies(dataset):
        preproc = preprocessing_dataframe(dataset)
        preproc_df = pd.DataFrame.from_dict(preproc)
        return preproc_df

    return get_preprocessed_sara_replies(dataset)


Preprocessing 2 - Separating paragraphs. Each paragraph is treated as a separate document. 
- Where double newline exists. Also found some areas with '\n whitespace \n' that has been replaced.
- As reply headers cannot be cleaned easily, the paragraph with To for example could be removed as this should just be the header. Likewise with empty documents. 
- The smallest ~1000 documents have length less than 20: these are usually letters/punctuation/name sign offs - these are less informative or useful in identifying the sensitivity of the information, so have been removed for simplicity in this preprocessing strategy.

In [5]:
def proc2(dataset):
    def process_text(input_text):
        clean = input_text
        clean = re.sub(r'=20', ' ', clean)
        clean = re.sub(r'=09', ' ', clean)
        # Also > Whitespace1+ > 
        clean = re.sub(r'\n\>', '\n', clean)
        clean = re.sub(r'\n\n\t', '\n', clean)
        # Replace whitespace between two newline characters with a single newline
        #cleaned_text = re.sub(r'\n\s*\n', '\n\n', input_text)
        #clean = re.sub(r'\n \n', '\n\n', clean)
        clean = re.sub(r'\n \n', '\n', clean)
        return clean

    def get_separate_paras(doc):
        paras = []
        payload = doc.split('\r\n\r\n')[1]
        payload = process_text(payload)
        split_load = payload.split('\n\n')
        for m in split_load:
            #if 'Subject:' in t:
            #    continue
            if 'To:' in m or 'From:' in m or 'mailto' in m or m == '':
                continue
            if len(m) < 2:
                continue
            if len(m) < 20:
                continue
            
            m = re.sub(r'\s+', ' ', m)
            paras.append(m)
        return paras

    def preproc(sample):
        sep_docs = []
        sep_docs_ids = []
        sep_docs_sens = []
        i = 0
        for s in sample.iterrows():

            separate_messages = get_separate_paras(s[1].text)
            for i, m in enumerate(separate_messages):
                id_part = s[1].doc_id + '_' + str(i)
                sep_docs_ids.append(id_part)
                sep_docs.append(m)
                sep_docs_sens.append(s[1].sensitivity)

        preproc = {}
        preproc['doc_id'] = sep_docs_ids
        preproc['text'] = sep_docs
        preproc['sensitivity'] = sep_docs_sens
        preproc_df = pd.DataFrame.from_dict(preproc)
        return preproc_df

    def get_preprocessed_sara_paras(dataset):
        preproc_data = preproc(dataset)
        preproc_df = pd.DataFrame.from_dict(preproc_data)
        return preproc_df

    return get_preprocessed_sara_paras(dataset)

Preprocessing 3

In [6]:
def proc3(dataset):
    def get_orig(message, find_orig_message):
        #line_start = message.rfind('\n', 0, find_orig_message)
        line_start = find_orig_message
        base_message = message[:line_start]
        line_end = find_orig_message + 5 #message.find('\n', find_orig_message)
        orig_message = message[line_end:]
        if 'Subject' in orig_message:
            over_header = orig_message.find('Subject:')
            orig_message = orig_message[over_header:]
        return base_message, orig_message

    def get_separate_messages(message):
        payload = message.split('\r\n\r\n')[1]
        separate_messages = []

        find_fwd_message = payload.find('- Forw')
        find_orig_message = payload.find('-----Original Message-')
        
        n = payload
        while (find_fwd_message > 0 or find_orig_message > 0):
            if (find_fwd_message > 0 and find_orig_message > 0):
                if find_fwd_message < find_orig_message:
                    b, n = get_orig(n, find_fwd_message)
                else:
                    b, n = get_orig(n, find_orig_message)
            elif (find_fwd_message > 0 and find_orig_message < 0):
                b, n = get_orig(n, find_fwd_message)
            else:
                b, n = get_orig(n, find_orig_message)
            
            find_fwd_message = n.find('- Forw')
            find_orig_message = n.find('-----Original Message-')

            b = re.sub(r'\s+', ' ', b)
            if len(b) < 20:
                continue
            separate_messages.append(b)

        n = re.sub(r'\s+', ' ', n)
        if 'Subject' in n:
            over_header = n.find('Subject:')
            n = n[over_header:]
        if len(n) > 20:
            separate_messages.append(n)

        return separate_messages

    def preprocessing_dataframe(testing_sample):
        ids = []
        texts = []
        sens = []
        preproc = {}

        for s in testing_sample.iterrows():
            separate_messages = get_separate_messages(s[1].text)

            if len(separate_messages) == 1:
                ids.append(s[1].doc_id)
                texts.append(separate_messages[0])
                sens.append(s[1].sensitivity)
            else:
                for i, m in enumerate(separate_messages):
                    id_part = s[1].doc_id + '_' + str(i)
                    ids.append(id_part)
                    texts.append(m)
                    sens.append(s[1].sensitivity)

        preproc['doc_id'] = ids
        preproc['text'] = texts
        preproc['sensitivity'] = sens
        return preproc

    def get_preprocessed_sara_p3(dataset):
        preproc = preprocessing_dataframe(dataset)
        preproc_df = pd.DataFrame.from_dict(preproc)
        return preproc_df

    return get_preprocessed_sara_p3(dataset)

### Llama 2 with SARA

In [7]:
def get_llama2(name):
    access_token = l2_token
    model_name = name
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, token=access_token)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", token=access_token, cache_dir=my_cache, revision='gptq-8bit-64g-actorder_True')
    return tokenizer, model

def get_model_version(m, v):
    models_dict = {'get_llama2' : get_llama2}
    return models_dict.get(m)(v)


In [43]:
def llm_inference(document, prompt, model, tokenizer):
  inputs = tokenizer(prompt(document), return_tensors='pt')
  generation_config = GenerationConfig(
    # Unable to set temperature to 0 - https://github.com/facebookresearch/llama/issues/687 - use do_sample=False for greedy decoding
    do_sample=False,
    max_new_tokens=20,
  )
  output = model.generate(inputs=inputs.input_ids.cuda(), attention_mask=inputs.attention_mask.cuda(), generation_config=generation_config)
  return tokenizer.decode(output[0], skip_special_tokens=True)

def display_gen_text(output, prompt_end):
  end_template = output.find(prompt_end)
  return output[end_template:]

def prompt_to_reply(d, p, model, tokenizer, end):
  response = llm_inference(d, p, model, tokenizer)
  return display_gen_text(response, end)


# String matching on model response
def post_process_classification(classification, ground_truth):
    if 'non-sensitive' in classification.lower():
        if ground_truth == 0:
            return 'TN', 0
        else:
            return 'FN', 0

    elif 'sensitive' in classification.lower() and 'non-sensitive' not in classification.lower():
        if ground_truth == 1:
            return 'TP', 1
        else:
            return 'FP', 1

    else:
        # Further processing required
        return classification, None
        further_processing_required[sample[1].doc_id] = classification


def clear_memory():
    # Prevents cuda out of memory
    torch.cuda.empty_cache()
    gc.collect()


# Dataset - dataframe, prompt_strategy - prompt function name
def llm_experiment(dataset, prompt_strategy, model, tokenizer, end_prompt=None):
    predictions = {
        'TP' : 0, # Sensitive
        'FP' : 0, # Non-sensitive document classified as sensitive
        'TN' : 0, # Non-sensitive
        'FN' : 0,
    }
    # Model output is not an expected sensitivity attribute
    further_processing_required = {}
    # All model output
    model_responses = {}

    scikit_true = []
    scikit_pred = []

    for sample in dataset.iterrows():
        sample_text = sample[1].text
        ground_truth = sample[1].sensitivity

        # To replace with appropriate pre-processing
        if len(sample_text) > 12000:
            continue
        
        classification = prompt_to_reply(sample_text, prompt_strategy, model, tokenizer, end_prompt)
        model_responses[sample[1].doc_id] = classification

        quadrant, pred = post_process_classification(classification, ground_truth)
        if pred == None:
            further_processing_required[sample[1].doc_id] = quadrant
            continue

        predictions[quadrant] = predictions.get(quadrant) + 1
        scikit_true.append(ground_truth)
        scikit_pred.append(pred)

        #clear_memory()

    return predictions, further_processing_required, model_responses, scikit_true, scikit_pred


In [9]:
def post_process_split_docs(mr, fpr, pre, df):
    clean_doc_id = {}
    ground_truths = []
    ite = -1
    for s in mr.keys(): #samp.doc_id():
        if s in fpr.keys():
            continue

        if '_' in s:
            s = s[:s.find('_')]

        val = clean_doc_id.get(s, -1)
        ite += 1

        if val == -1:
            clean_doc_id[s] = pre[ite]
            ground_truths.append((df[df.doc_id == s].sensitivity).iloc[0])
            continue

        if (val == pre[ite] or val == 1):
            continue
        
        clean_doc_id[s] = pre[ite]

    values_array = np.array(list(clean_doc_id.values()))
    #print(ground_truths)
    #print(values_array)
    return values_array, ground_truths


### Evaluation summary

In [10]:
def basic_metrics(true_labels, predictions):
    acc = accuracy_score(true_labels, predictions)
    bac = balanced_accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average=None)


def evaluation_summary(description, true_labels, predictions):
    target_classes = ['Non-sensitive (0)', 'Sensitive (1)']
    print("Evaluation for: " + description)
    print(classification_report(true_labels, predictions, digits=3, zero_division=0, target_names=target_classes))
    print('\n\nConfusion matrix:')
    confusionMatrix = confusion_matrix(true_labels, predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=confusionMatrix, display_labels=target_classes) 
    disp.plot()
    plt.show()

def get_metric_dict(method, labels, preds):
    acc = accuracy_score(labels, preds)
    bac = balanced_accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    
    metric_dict = {'accuracy': acc, 'f1_score': f1, 'balanced accuracy': bac}
    print(metric_dict)

 
#evaluation_summary("Llama-2", scikit_true, scikit_pred)

### Prompts

In [11]:
# Init
def base(document):
  return f"""[INST]
The Text is sensitive if it contains sensitive personal information.
Which one of the attributes “sensitive”, “non-sensitive” classifies the Text. 
The Text is delimited by triple backticks. You must answer after 'Attribute:'[/INST]

Text: ```{document}```.

Attribute:"""

def b1(document):
  return f"You are identifying documents containing personal sensitive information. Is the given text classified as 'sensitive' or 'non-sensitive'? You must answer with a the class inside a Python list and provide no further explanation, for example ['non-sensitive']. \nText: {document} \n"

def b2(document):
  return f"You are identifying documents containing personal sensitive information. Is the given text classified as 'sensitive' or 'non-sensitive'? You must answer with a the class inside a Python list and provide no further explanation, for example ['sensitive']. \nText: {document} \n"

def b3(document):
  return f"You are identifying documents containing personal sensitive information. \nText: {document} \nWhich one of the attributes 'sensitive', 'non-sensitive' describes a given text? You must answer with a Python list containing the only appropriate attribute and provide no further explanation. \n"



### Main experiment - main call

In [23]:
# Get dataset
# Preprocess
# Get model
# Get prompt
# Run evaluation
# Display evaluation metrics

def main_experiment(portion, portion_value, proc_meth, m, v, prompt_strategy, end_prompt, tokenizer, model):
    sara_dataset = get_sara()
    sara_df = dataset_to_df(sara_dataset)
    if portion == 'n':
        samp = get_sample_n(sara_df, portion_value)
    else:
        samp = get_sample_frac(sara_df, portion_value)

    if proc_meth == 'replies':
        proc_samp = proc1(samp)
    elif proc_meth == 'replies_nometa':
        proc_samp = proc3(samp)
    else:
        proc_samp = proc2(samp)
        
    #tokenizer, model = get_model_version(m, v)
    predictions, further_processing_required, model_responses, truths, preds = llm_experiment(proc_samp, prompt_strategy, model, tokenizer, end_prompt)
    new_preds, new_truths = post_process_split_docs(model_responses, further_processing_required, preds, sara_df)
    #evaluation_summary("Llama-2", truths, preds)
    #get_metric_dict("Llama-2", truths, preds)
    print(model_responses)
    #evaluation_summary("Llama-2", new_truths, new_preds)
    #get_metric_dict("Llama-2", new_truths, new_preds)
    return model_responses

In [13]:
m = 'get_llama2'
v = 'TheBloke/Llama-2-13B-chat-GPTQ' #"meta-llama/Llama-2-7b-chat-hf"
tokenizer, model = get_model_version(m, v)

CUDA extension not installed.
CUDA extension not installed.


In [ ]:
def llm_inference(document, prompt, model, tokenizer):
    messages = [
    {"role": "user", "content": prompt(document)},
    ]
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    device = 'cuda'
    model_inputs = encodeds.to(device)
    arr_like = torch.ones_like(model_inputs)
    attention_mask = arr_like.to(device)
    generated_ids = model.generate(inputs=model_inputs, attention_mask=attention_mask, max_new_tokens=30, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    return decoded[0]

Code not present for the experiments with above chat template. However, we found:
* The 70B model seems to answer most sensibly with only lists but can only operate on smaller documents (length roughly 1000). Whereas, the 7B models seem to not follow the appropriate template pattern. We primarily worked with the 13B 8 bit model.
* The chat template adds instruction tags. Attempted giving assistant role with content document, with prompt surrounding it from user role as instruction commands. Using a single user role with entire prompt in content seems to give the most sensible output.
* We requested a Python list output, following the literature.
* We also found some samples are empty strings from inspecting the models output when it requested for the text. We add a statement to exclude small strings from the sample during preprocessing.

Further chat template experiments:

In [44]:
def b1(document):
  return f"You are identifying documents containing personal sensitive information. Is the given text classified as 'sensitive' or 'non-sensitive'? \nText: {document} "

In [45]:
def llm_inference(document, prompt, model, tokenizer):
    messages = [
    {"role": "system", "content": "You must answer with a Python list containing the only appropriate class and provide no further explanation."},
    {"role": "user", "content": prompt(document)},
    #{"role": "assistant", "content": "\n["}
    ]
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    device = 'cuda'
    model_inputs = encodeds.to(device)
    arr_like = torch.ones_like(model_inputs)
    attention_mask = arr_like.to(device)
    generated_ids = model.generate(inputs=model_inputs, attention_mask=attention_mask, max_new_tokens=30, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    #print(decoded)
    return decoded[0]

In [48]:
#mr = main_experiment('n', 30, 'replies_nometa', 'get_llama2', "meta-llama/Llama-2-7b-chat-hf", b1, '[/INST]', tokenizer, model)

### Mistral

In [9]:
def llm_inference(document, prompt, model):
  messages = [
    {"role": "user", "content": get_prompt_template(document, prompt)}
  ]
  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
  inputs = encodeds.to('cuda')
  ### Array like
  arr_like = torch.ones_like(inputs)
  attention_mask = arr_like.to('cuda')
  generated_ids = model.generate(inputs=inputs, attention_mask=attention_mask, max_new_tokens=150, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0]

device = "cuda"
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", cache_dir=my_cache)

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.62s/it]
